# 🌤 Weather Temperature

## Libraries

* Let's import the usual libraries:

In [1]:
# Data manipulation
import numpy as np
import pandas as pd
pd.set_option("max_columns", None)

# Data Visualiation
import matplotlib.pyplot as plt
import seaborn as sns

# System
import os

# Deep Learning
import tensorflow as tf

* Manipulating temporal data is tricky, let's also import 📚 [`typing`](https://docs.python.org/3/library/typing.html) to check the types of variables we will be dealing with in our Python functions:

In [2]:
from typing import Dict, List, Tuple, Sequence

## (0) The weather temperature challenge

### (0.0) Introduction

🧑🏻‍🏫 **Goals:**
- Prepare a dataset to be fed into a Recurrent Neural Network
- Develop a better understanding of Time Series

❗️ **Warning/Disclaimer**:
- This challenge is truly designed to help you understand **how to deal with temporal data**, using an LSTM architecture as a _tool_, not to focus on the different gates of the LSTM or designing the "best" recurrent network.

🎯 **ML target**:
* In this challenge, we want to predict the **temperature in the next 3, 6, 9, 12... hours**... 
* ...based on a sequence of weather features such as the _past temperature_, the _atmospheric pressure_, the _humidity_, etc..

### (0.1) The weather dataset

#### (0.1.1) Loading the dataset

🌤 This challenge uses a [**weather time series dataset**](https://www.bgc-jena.mpg.de/wetter/) recorded by the [**Max-Planck-Institute for Biogeochemistry**](https://www.bgc-jena.mpg.de/index.php/Main/HomePage). This dataset contains $14$ different features such as _air temperature_, _atmospheric pressure_ and _humidity_ that were collected starting in 2003 every 10 minutes (~ 420k rows). But for efficiency, you will use "only" the data collected between 2009 and 2016 every three hours. Indeed, this time interval seems reasonable to observe the evolution of the temperature throughout a given day.

🛠 We've already performed the following feature engineering steps for you:
- taking every $18$th record to focus on predictions every three hours $ ( 18 =  \frac{6 records}{hour} \times 3 hours)$
- replacing absurd values
- _wind_: computing the wind directions as wind vectors with coordinates (`Wx`, `Wy`)
- computing _the daily and yearly periodicities_, stored through (`Day sin`, `Day cos`) and (`Year sin`, `Year cos`)

##### (Keep this section for later) Downloading the original dataset and engineer the features manually

_(Toggle the section to hide it for the moment)_

🥋 If you want to practice your feature engineering skills, feel free to download the original dataset and work on it (but not today, there are many questions and reading sections to cover 😉)

In [3]:
#########################################################
# Uncomment later -not today- to download the original  #
# dataset and try to perform the features engineering   #
# by yourself                                           #
#########################################################


zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

df = pd.read_csv(csv_path)

print(f"df.shape = {df.shape}")
display(df.head())

In [4]:
# Inspecting each feature to detect their type and null values
df.info()

In [5]:
# Convert the "Date Time" column to a datetime format

pass  # YOUR CODE HERE

In [6]:
# Slice [start:stop:step], 
# starting from index 5, take every 6th record
# to get only hourly records

pass  # YOUR CODE HERE

In [7]:
# Describe the dataset

pass  # YOUR CODE HERE

In [8]:
# Replacing the absurd values
pd.options.mode.chained_assignment = None  # default='warn'

## Fixing the wv

pass  # YOUR CODE HERE

## Fixing the max wv

pass  # YOUR CODE HERE

In [9]:
# Wind : Wd degrees from 0 to 360 egrees
# Angles do not make good models inputs because 0 and 360 should be "close"

print('-'*50)
print("Working with angles...")

plt.hist2d(df['wd (deg)'], df['wv (m/s)'], 
           bins=(50, 50), 
           vmax=400)
plt.colorbar()
plt.xlabel('Wind Direction [deg]')
plt.ylabel('Wind Velocity [m/s]')
plt.show()

# It is much easier for the model to interpret
# the wind direction and the wind velocity through a vector


# Convert degrees to radians and store the values into wd_rad
pass  # YOUR CODE HERE

# Calculate the wind x and y components and store then in two new columns
# `Wx` and `Wy`
pass  # YOUR CODE HERE

# Calculate the max wind x and y components and store then in two new columns
# `max Wx` and `max Wy`
pass  # YOUR CODE HERE

print('-'*50)
print("Working with wind vectors")

plt.hist2d(df['Wx'], df['Wy'], 
           bins=(50, 50), vmax=400)
plt.colorbar()
plt.xlabel('Wind X [m/s]')
plt.ylabel('Wind Y [m/s]')
ax = plt.gca()
ax.axis('tight')
plt.show()

# $CHALLENGIFY_END

In [10]:
# Similarly to the wind direction, the time in seconds is not a useful model input
# The weather dataset has clear daily and yearly periodicities
# Using sine and cosine functions, we can compute:
# - the time of the day
# - the time of the year

pass  # YOUR CODE HERE

In [11]:
# Select every 3 hours 

pass  # YOUR CODE HERE

##### Trust us and start from this already preprocessed dataset for this challenge

In [12]:
url = "https://wagon-public-datasets.s3.amazonaws.com/deep_learning_datasets/weather_every_three_hours_engineered.csv"
df = pd.read_csv(url).drop(columns = ['Unnamed: 0'])
df

👆 In the preprocessed dataset, we have :
- $23$k rows  (~ 8 years of weather data)
- $19$ features composed of:
    - $1$ <font color=green>**target**</font> (we will use the past values of the temperature as a feature)
    - $18$ <font color=orange>**past covariates**</font> (= features which past values are known)
    - $0$ <font color=blue>**future covariates**</font> (= features which future values are known, e.g. public holidays)

    
<img src='https://github.com/lewagon/data-images/blob/master/DL/time-series-covariates.png?raw=true'>

👨🏻‍🏫 This weather dataset is a DataFrame (dimension = 2) which is a single Time Series from the beginning of 2009 to the end 2016 with records every 3 hours. 

* `df.shape = (n_timesteps, n_features) = (23363, 19)`

🎯 The goal is to predict the temperature in 3, 6, 9, 12, ... hours using the past values.

In [13]:
TARGET = 'T (degC)'

#### (0.0.2) Visualising your Time Series

📈  Here is the ***evolution of some features over time***:
* `T (degC)` (temperature)
* `p (mbar)` (atmospheric pressure)
* `rho (g/m**3)` (atmospheric density)



In [14]:
plot_cols = [TARGET, 'p (mbar)', 'rho (g/m**3)']
plot_features = df[plot_cols]
plot_features.index = df.index
plot_features.plot(subplots = True);

## (1) Prepare the dataset

###  (1.0) The big picture

<b><u>Step 1: Cross-Validation in Time Series [FOLDS] </u></b>

* Starting from this single Time Series, we will create <font color="#c91ac9">**FOLDS**</font>...
* ... and train/evaluate our LSTM on these different <font color="#c91ac9">**FOLDS**</font> to conclude about the robustness of the neural network

<b><u>Step 2: Holdout method within each fold [TRAIN-TEST SPLIT]</u></b>

* For each <font color="#c91ac9">**FOLD**</font>, we will do a TRAIN-TEST SPLIT to:
    * <font color=blue>**fit**</font> the model on the <font color=blue>**train**</font> set 
    * and <font color="#ff8005">**evaluate**</font> it on the <font color="#ff8005">**test**</font> set

👇 The first two steps can be summarized in the following image:

<img src="https://bit.ly/3yLoa92" alt="Time Series Cross Validation" width="500" height="500">


<b><u>Step 3: Sampling SEQUENCES in both the train set and the test set</u></b>

In each <font color=blue>**train**</font> set and each <font color="#ff8005">**test**</font> set, we will create <font color=magenta>**random sequences**</font> as illustrated down below 👇:

<img src="https://bit.ly/3Ri8Vfd" alt="Sequences in each fold" width="500" height="500"> 



### (1.1) Creating FOLDS to cross-validate a Time Series, each of them with shape `(n_timesteps_per_fold, n_features)` 

✂️ As for any Machine Learning / Deep Learning model, you are strongly advised to perform a **cross-validation** of your model:

* `fold_1.shape = (n_timesteps_per_fold, n_features)`
* `fold_2.shape = (n_timesteps_per_fold, n_features)`
* ...
* `fold_K.shape = (n_timesteps_per_fold, n_features)`

ℹ️ The number of steps `n_timesteps_per_fold` is also called the `fold_length`.

> ☢️ ***In order to <u>avoid data leakage in Time Series</u>, always <u>split</u> the <font color=blue>train</font> set <u>chronologically</u> before the <font color="#ff8005">test</font> set!***

👇 _Here is an example of a { 4-Fold cross-validation } + { train-test split for each fold } that we just saw earlier_:
<img src="https://bit.ly/3yLoa92" alt="Time Series Cross Validation" width="500" height="500"> 

* We usually create as many folds as needed to clearly test all types of past conditions,
    * e.g. crash market periods 📉, extreme increases in temperature 📈, atone markets 😴, etc...
* It is very common to create ***hundreds of folds*** in Time Series forecasting.

---

🌐 Let's define some global variables that we will use for our tests everywhere in this notebook:

In [15]:
# --------------------------------------------------- #
# Let's consider FOLDS with a length of one year      #
# --------------------------------------------------- #

FOLD_LENGTH = 8*365 # every 3 hrs x 8 = 24h
                    # one year            

# --------------------------------------------------- #
# Let's consider FOLDS starting every two weeks       #
# --------------------------------------------------- #
    
FOLD_STRIDE = 8*14 # every 3 hrs x 8 = 24h
                   # 2 weeks = 14 days

# --------------------------------------------------- #
# Let's consider a train-test-split ratio of 70/30    #
# --------------------------------------------------- #

TRAIN_TEST_RATIO = 0.7

❓ **Question (<font color="#c91ac9">FOLDS</font>)** ❓

Code the function `get_folds` that:
- <i>(input)</i> given a Time Series  (in a form of a DataFrame `df` with timesteps as indexes and features as columns), a `fold_length` and a `fold_stride`
- <i>(output)</i> returns all the possible folds.

In [16]:
def get_folds(
    df: pd.DataFrame, 
    fold_length: int,
    fold_stride: int) -> List[pd.DataFrame]:
    '''
    This function slides through the Time Series (a DataFrame with features in columns and timesteps in indexes)
    to create folds of equal length (through the fold_length argument),
    using `fold_stride` between each fold.
    
    It returns a list of folds, each as a DataFrame with features in columns and timesteps in indexes
    (you can think about these DataFrames as 2D-array)
    '''
    pass  # YOUR CODE HERE

🧪 ***Test your code***

Run the cell down below, if no error message appears, you can move forward!

In [17]:
# Creating folds based on the 23k rows, 1 year of fold_length and 14 days of fold_stride
folds = get_folds(df, FOLD_LENGTH, FOLD_STRIDE)
assert(len(folds)==183)

### (1.2) Temporal Train/Test Split

👩🏻‍🏫 Let's focus on one fold for the moment.

In [18]:
fold = folds[0]
fold

> ☢️ ***In order to <u>avoid data leakage in Time Series</u>, always <u>split</u> the <font color=blue>train</font> set <u>chronologically</u> before the <font color="#ff8005">test</font> set!***


❗️ The index for the last <font color="blue">*y_train*</font> should be just before the index of the first <font color="#ff8005">*y_test*</font> 

❗️ After splitting this fold into a <font color="blue">train</font> set and a <font color="#ff8005">*test*</font> set, we will sample sequences of length equal to 2 weeks (it's quite a common period for weather forecasting). 

In [19]:
INPUT_LENGTH = 8 * 14 # records every 3 hours x 8 = 24 hours
                      # two weeks

❓ **Question (temporal train-test split)** ❓

Code the function `train_test_split` down below which:
- <i>(input)</i> given a Time Series  (in a form of a DataFrame `df` with timesteps as indexes and features as columns), a `train_test_ratio` and an `input_length`
- <i>(output)</i> a `fold_train` and a `fold_test`

Don't forget to take into account the two aforementioned warnings (❗️) and the following illustration:

<img src="https://github.com/lewagon/data-images/blob/master/DL/explanations_for_train_test_split_temporal.png?raw=true" alt="train_test_split_temporal" width="500" height="500"> 

In [20]:
def train_test_split(fold:pd.DataFrame,
                     train_test_ratio: float,
                     input_length: int) -> Tuple[pd.DataFrame]:
    '''
    Returns a train dataframe and a test dataframe from which one can sample (X,y) sequences.
    df_train should contain all the timesteps until round(train_test_ratio * len(fold))   
    '''
    # $CHALLENGIFY_BEGIN
    
    # TRAIN SET
    last_train_idx = round(train_test_ratio * len(fold))
    fold_train = fold.iloc[0:last_train_idx, :]

    # TEST SET
    first_test_idx = last_train_idx - input_length
    fold_test = fold.iloc[first_test_idx:, :]

    return (fold_train, fold_test)

🧪 ***Test your code***

Run the cell down below, if no error message appears, you can move forward!

In [21]:
(fold_train, fold_test) = train_test_split(fold, TRAIN_TEST_RATIO, INPUT_LENGTH)

assert(fold_train.index.start == 0)
assert(fold_train.index.stop == 2044)
assert(fold_test.index.start == 1932)
assert(fold_test.index.stop == 2920)

## (1.3) Create (X, y) sequences

🗓 Now that we have splitted our fold into a <font color="blue">train</font> set and a <font color="#ff8005">test</font> set, it is time to:
- 🏋 sample sequences on which the model will be <font color="blue">trained</font>
- 👩🏻‍🏫 sample sequences on which the model will be <font color="#ff8005">evaluated</font>


💡 Note that each sequence will have:
1. a shape `(n_observations, target)`
2. $(X,y)$ where:

- $X$ corresponds to:
    - The <font color=green>past values of the **target**</font> 
    - The values of the $18$ <font color=orange>**past covariates**</font> (= features which past values are known)
    - We don't have any <font color=blue>**future covariates**</font> here (= features which future values are known, e.g. public holidays)
    
- $y$ is the <font color=green>**target**</font> that we want to predict in 3 hours, i.e. the value at the next timestep
    - It could also be several values in the future, 3 hours later, 6 hours later, 9 hours later, ... for let's keep it simple here and just try to predict the next point (in 3 hours)
    - We will denote the number of values we want to predict in the future as the `output_length`

In [22]:
OUTPUT_LENGTH = 1

<img src="https://bit.ly/3Ri8Vfd" alt="Sequences in each fold" width="500" height="500"> 

🎯 In this section, the goal is to create `(X_train, y_train)` and `(X_test, y_test)` containing all the SEQUENCES you need to train and test your model for this fold:

* `X_train.shape = (n_samples_train, input_length, n_covariate_features)`
* `y_train.shape = (n_samples_train, output_length, n_targets)`

👉 Notice that we are now dealing with *3D arrays* instead of *2D arrays* (`fold_train` and `fold_test` were *Pandas DataFrame*, hence bi-dimensional)

<img src="https://bit.ly/3bOhKNj" alt="3d arrays time series" width="1200" height="800"> 

💡 To create these SEQUENCES within the train set and the test set, you have several options, among them:
- 🎲 <u><i>Option 1</i></u>: creating these sequences by randomly sampling $(X_i, y_i)$ from <font color="blue">fold_train</font> and <font color="#ff8005">fold_test</font>.
- ⌚️ <u><i>Option 2</i></u>: scanning a fold chronologically
 

👉 Let's focus on the first option.

🧠 After creating sequences within the train set and the test set, we will:
- evaluate a baseline model (this _baseline model_ will be explained in the next section `(2) Modelling`)
-  <font color="blue">train</font> and <font color="#ff8005">evaluate</font> an LSTM

🎁 If you want to scan the folds chronologically, we provided the solution in the section (1.2.2) and you can come back to it later during the projects or after the bootcamp.


#### (1.2.1) Option 1: Random sampling

👇 We will code:

* 1️⃣ a function `get_Xi_yi` to generate a single sequence from a fold

* 2️⃣ a function `get_X_y` to generate multiple sequences from a fold, calling the first function `get_Xi_yi`.

##### (1.2.2.1) Generating one random sequence

<img src="https://github.com/lewagon/data-images/blob/master/DL/get_xi_yi.png?raw=true" alt="one sequence" width="400" height="400"> 

❓ **Question (extracting a random sequence from a fold)** ❓

Code the function `get_Xi_yi` down below which:
- <i>(input)</i> given a fold, an `input_length` and an `output_length`
- <i>(output)</i> returns a sequence $(X_i,y_i)$

In [23]:
def get_Xi_yi(
    fold:pd.DataFrame, 
    input_length:int, 
    output_length:int):
    '''
    - given a fold, it returns one sequence (X_i, y_i)
    - with the starting point of the sequence being chosen at random
    '''
    pass  # YOUR CODE HERE
    return X_i, y_i

🧪 ***Test your code***

Run the cell down below, if no error message appears, you can move forward!

In [24]:
X_train_i, y_train_i = get_Xi_yi(fold_train, INPUT_LENGTH, OUTPUT_LENGTH)
X_test_i, y_test_i = get_Xi_yi(fold_test, INPUT_LENGTH, OUTPUT_LENGTH)

assert(X_train_i.shape==(112,19))
assert(y_train_i.shape==(1,1))
assert(X_test_i.shape==(112,19))
assert(y_test_i.shape==(1,1))

##### (1.2.2.2) Generating multiple random sequences

<img src="https://bit.ly/3Ri8Vfd" alt="Sequences in each fold" width="500" height="500"> 

❓ **Question (extracting multiple random sequence from a fold)** ❓

Code the function `get_X_y` down below which:
- <i>(input)</i> given a fold, a `number_of_sequences` an `input_length` and an `output_length`
- <i>(output)</i> returns $(X,y)$ 

_Don't forget to use the `get_Xi_yi` function that you have just coded!_

In [25]:
def get_X_y(
    fold:pd.DataFrame,
    number_of_sequences:int,
    input_length:int,
    output_length:int
):
    pass  # YOUR CODE HERE
    return np.array(X), np.array(y)

🧪 ***Test your code***

Run the cell down below, if no error message appears, you can move forward!

In [26]:
N_TRAIN = 700 # number_of_sequences_train
N_TEST = 300 # number_of_sequences_test

X_train, y_train = get_X_y(fold_train, N_TRAIN, INPUT_LENGTH, OUTPUT_LENGTH)
X_test, y_test = get_X_y(fold_test, N_TEST, INPUT_LENGTH, OUTPUT_LENGTH)

assert(X_train.shape==(N_TRAIN, INPUT_LENGTH, 19))
assert(y_train.shape==(N_TRAIN, OUTPUT_LENGTH, 1))
assert(X_test.shape==(N_TEST, INPUT_LENGTH, 19))
assert(y_test.shape==(N_TEST, OUTPUT_LENGTH, 1))

#### (1.2.2) (Read this section later) 🎁 Option 2: Scanning  chronologically

_(Toggle the section to hide it for the moment)_

As stated earlier, there are multiple ways to extract sequences from a fold. 

- 🎲 In the previous section, you coded:
    - `get_Xi_yi` which randomly samples _one_ sequence 
    - and `get_X_y` which randomly generates _multiple_ sequences

- ⌚️ In this section, we provide you a unique function `get_X_y_strides`.
    - It scans a fold chronologically based on:
         - an `input_length` (let's still use INPUT_LENGTH = 8 * 14, i.e. two weeks) 
         - and a `sequence_stride` (think about a one-dimensional convolutional operation!)

👉 Let's scan the fold with a temporal window of one week:

In [27]:
SEQUENCE_STRIDE = 8 * 7 # 8 records every 3 hours x 7 days

In [28]:
def get_X_y_strides(fold: pd.DataFrame, input_length: int, output_length: int, sequence_stride: int):
    '''
    - slides through a `fold` Time Series (2D array) to create sequences of equal
        * `input_length` for X,
        * `output_length` for y,
    using a temporal gap `sequence_stride` between each sequence
    - returns a list of sequences, each as a 2D-array time series
    '''

    X, y = [], []

    TARGET = 'T (degC)'

    for i in range(0, len(fold), sequence_stride):
        # Exits the loop as soon as the last fold index would exceed the last index
        if (i + input_length + output_length) > len(fold):
            break
        X_i = fold.iloc[i:i + input_length, :]
        y_i = fold.iloc[i + input_length:i + input_length + output_length, :][[TARGET]]
        X.append(X_i)
        y.append(y_i)

    return np.array(X), np.array(y)

🧑🏻‍🎓 Some clarifications about scanning a fold sequentially :

In [29]:
# As a reminder, FOLD_LENGTH = 1 year

print("FOLD_LENGTH") 
print(f"= {FOLD_LENGTH} timesteps")
print(f"= {int(FOLD_LENGTH/8)} days") # 8 records per day, every 3 hours
print(f"= {int(FOLD_LENGTH/8/7)} weeks")

In [30]:
# For one fold, if we scan it chronologically...

print("INPUT_LENGTH for each sequence") 
print(f"= {INPUT_LENGTH} timesteps")
print(f"= {int(INPUT_LENGTH/8)} days") # 8 records per day, every 3 hours
print(f"= {int(INPUT_LENGTH/8/7)} weeks")

In [31]:
print("OUTPUT_LENGTH for each sequence")
print(f"= {OUTPUT_LENGTH} timestep(s)")

<img src="https://github.com/lewagon/data-images/blob/master/DL/scanning_a_time_series_chronologically_v3.png?raw=true">

👆 In this illustration, there are 7 weeks. 

- As `SEQUENCE_STRIDE` $ = 1$ week $= 7$ days and `OUTPUT_LENGTH` $= 1$ day, the last possible index for the target is equal to $ 48 - 7 + 1 = 42$ (in terms of days, not timesteps of 3 hours)
- Since `INPUT_LENGTH` $ = 2$ weeks = $ 14$  days, the last possible starting index for a sequence is $42 - 14 = 28$.

🧪 ***Test your code*** (if you decide to try to code the function by yourself)

In [32]:
X_strides, y_strides = get_X_y_strides(fold, INPUT_LENGTH, OUTPUT_LENGTH, SEQUENCE_STRIDE)

assert(X_strides.shape==(51,112,19))
assert(y_strides.shape==(51,1,1))

## (2) Modelling

**The MAE as a metrics to monitor the temperature prediction**

The Mean Absolute Error seems to be a reasonable metrics to evaluate a model's capability to predict the temperature:

$$ MAE = \frac{1}{n_{samples}} \times \sum_{i = 1}^{n_{samples}} |y_{true}^{(i)} - y_{pred}^{(i)}|$$

### (2.1) 🎁  A baseline model for temporal data: the `Last Seen Value` 

👉 For a regression task, the baseline model usually consists in predicting the average value $\mu_{\color{blue}{train}}$ for all the values of $y_{\color{red}{pred}}$. 

❗️ However, in Time Series, this baseline model is _NOT_ relevant because of this intrinsic temporality and potentially cyclical patterns.

> 🧑🏻‍🏫 **An "intuitive" baseline model is to predict the last seen value for the future value(s) you want to forecast**, as illustrated down below!

<img src = "https://github.com/lewagon/data-images/blob/master/DL/rnn_time_series_no_horizon.png?raw=true" width = 600, height = 300>

In [33]:
print(f"In the `Create (X,y) sequences` section, we sampled {N_TRAIN} training sequences and {N_TEST} test sequences")

🎁 Let's code a function `last_seen_value_baseline_model` which:
- _(input)_ given ***multiple sequences***
- _(output)_ <font color="#dc143c">***computes for each sequence the last seen value as a prediction for the temperature for the next timestep(s) and returns the mean absolute error of the errors between the real values and these baseline predictions.***</font>

as illustrated in the picture down below 👇:

<img src = "https://github.com/lewagon/data-images/blob/master/DL/rnn_baseline_model_last_seen_value.png?raw=true" width = 600, height = 300>

In [34]:
def last_seen_value_baseline_model(X, y):
    '''
    This function returns the last seen temperature for each sequence
    and compute the MAE of this baseline model
    '''
    last_seen_values = X[:,-1,1] # : for all the sequences
                                 # -1 last timestep for each sequence
                                 # 1 for the temperature column as X is a NumPy array and not a DataFrame
            
    y = y.reshape(len(y),)

    errors = y - last_seen_values # real values - predicted values
    mean_absolute_error = np.mean(np.abs(errors))
    
    overview_baseline_errors = pd.DataFrame({"y":y,
                                            "last_seen_values":last_seen_values,
                                            "absolute_errors":np.abs(errors)})
    
    print("-"*50)
    print("Showing absolute errors for the first 5 sequences:")
    display(overview_baseline_errors.head())
    
    print("-"*50)
    print(f"The MAE on these set of sequences using the baseline model is equal to {round(mean_absolute_error,2)} Celsius degrees")
    
    return mean_absolute_error

🎁 Let's evaluate our baseline model on the <font color="#">test</font> set of the `fold[0]`

In [35]:
mean_absolute_error = last_seen_value_baseline_model(X_test, y_test)

In [36]:
print(f"This MAE of {round(mean_absolute_error,2)} Celsius degrees was obtained for the fold[0]")

### (2.2) A Recurrent Neural Network: the `LSTM`

🚀 It is time to design a Recurrent Neural Network and hopefully beat the baseline 💪 !

❓ **Question (RNN)** ❓ 

- Create a function `init_model` which builds and compiles a simple Recurrent Neural Network with an LSTM layer

In [37]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, metrics
from tensorflow.keras.layers.experimental.preprocessing import Normalization

def init_model(X_train, y_train):
    
    pass  # YOUR CODE HERE

🛠 🎁 📉 We coded a function `plot_history` for you to visualize the training of your RNN over epochs. This function shows both the evolution of the loss function (MSE) and metrics (MAE).

In [38]:
def plot_history(history):
    
    fig, ax = plt.subplots(1,2, figsize=(20,7))
    # --- LOSS: MSE --- 
    ax[0].plot(history.history['loss'])
    ax[0].plot(history.history['val_loss'])
    ax[0].set_title('MSE')
    ax[0].set_ylabel('Loss')
    ax[0].set_xlabel('Epoch')
    ax[0].legend(['Train', 'Test'], loc='best')
    ax[0].grid(axis="x",linewidth=0.5)
    ax[0].grid(axis="y",linewidth=0.5)
    
    # --- METRICS:MAE ---
    
    ax[1].plot(history.history['mae'])
    ax[1].plot(history.history['val_mae'])
    ax[1].set_title('MAE')
    ax[1].set_ylabel('MAE')
    ax[1].set_xlabel('Epoch')
    ax[1].legend(['Train', 'Test'], loc='best')
    ax[1].grid(axis="x",linewidth=0.5)
    ax[1].grid(axis="y",linewidth=0.5)
                        
    return ax

❓ **Question (Training and evaluating)** ❓

- Initialize an RNN model with the `init_model` function
- Train it
- Evaluate it

In [39]:
# YOUR CODE HERE

In [40]:
# 2 - Evaluation
# ====================================
res = model.evaluate(X_test, y_test)
print(f"The MAE on the test set is equal to {round(res[1],2)} Celsius degrees")

## (3) 🎁 Cross-Validation

Do you remember the section **`(1.0) The big picture`** ? 

If not, don't scroll up, the notebook is quite long, here it is 👇:

<hr>

<b><u>Step 1: Cross-Validation in Time Series [FOLDS] </u></b>

* Starting from this single Time Series, we will create <font color="#c91ac9">**FOLDS**</font>...
* ... and train/evaluate our LSTM on these different <font color="#c91ac9">**FOLDS**</font> to conclude about the robustness of the neural network

<b><u>Step 2: Holdout method within each fold [TRAIN-TEST SPLIT]</u></b>

* For each <font color="#c91ac9">**FOLD**</font>, we will do a TRAIN-TEST SPLIT to:
    * <font color=blue>**fit**</font> the model on the <font color=blue>**train**</font> set 
    * and <font color="#ff8005">**evaluate**</font> it on the <font color="#ff8005">**test**</font> set

👇 The first two steps can be summarized in the following image:

<img src="https://bit.ly/3yLoa92" alt="Time Series Cross Validation" width="500" height="500">


<b><u>Step 3: Sampling SEQUENCES in both the train set and the test set</u></b>

In each <font color=blue>**train**</font> set and each <font color="#ff8005">**test**</font> set, we will create <font color=magenta>**random sequences**</font> as illustrated down below 👇:

<img src="https://bit.ly/3Ri8Vfd" alt="Sequences in each fold" width="500" height="500"> 

<hr>

❗️ **WARNING**❗️

- Keep in mind that we did steps 2 and 3 for one single fold. 
- If we want to ensure the robustness of a model, we need to cross-validate the model on ALL the folds!

In [41]:
print(f"WARNING, we have {len(folds)} FOLDS, so you'd better run the cross-validation of the baseline model and the RNN on Colab...")

In [42]:
# Reminders of the global variables in this notebook

FOLD_LENGTH = 8*365 # every 3 hrs x 8 = 24h
                    # one year  
    
FOLD_STRIDE = 8*14 # every 3 hrs x 8 = 24h
                   # 2 weeks = 14 days
    
TRAIN_TEST_RATIO = 0.7

INPUT_LENGTH = 8 * 14 # records every 3 hours x 8 = 24 hours
                      # two weeks

OUTPUT_LENGTH = 1 # predict the temperature in 3 hours, a.k.a next timestep

In [43]:
from tensorflow.keras.callbacks import EarlyStopping

def cross_validate_baseline_and_lstm(folds):
    '''
    This function cross-validates 
    - the "last seen value" baseline model
    - the RNN model
    '''
    
    list_of_mae_baseline_model = []
    list_of_mae_recurrent_model = []
    
    for fold in folds:
        
        # 1 Train/Test split the current fold
        (fold_train, fold_test) = train_test_split(fold, TRAIN_TEST_RATIO, INPUT_LENGTH)                   


        # 2 - Generate sampled sequences in both the train set and the test set
        N_TRAIN = 700 # number_of_sequences_train
        N_TEST = 300 # number_of_sequences_test

        X_train, y_train = get_X_y(fold_train, N_TRAIN, INPUT_LENGTH, OUTPUT_LENGTH)
        X_test, y_test = get_X_y(fold_test, N_TEST, INPUT_LENGTH, OUTPUT_LENGTH)

        # 3 - Baseline Model for this fold
        mae_baseline = last_seen_value_baseline_model(X_test, y_test)
        list_of_mae_baseline_model.append(mae_baseline)

        # 4- LSTM MODEL
        model = init_model(X_train, y_train)
        es = EarlyStopping(monitor = "val_mae",
                           mode = "min",
                           patience = 10, 
                           restore_best_weights = True)
        history = model.fit(X_train, y_train,
                            validation_split = 0.2,
                            shuffle = False,
                            batch_size = 8,
                            epochs = 50,
                            callbacks = [es],
                            verbose = 1)
        res = model.evaluate(X_test, y_test)
        mae_lstm = res[1]
        list_of_mae_recurrent_model.append(mae_lstm)

    return np.mean(list_of_mae_baseline_model), np.mean(list_of_mae_recurrent_model)

In [ ]:
# On Colab
# mae_baseline_model_cv, mae_lstm_model_cv = cross_validate_baseline_and_lstm(folds)

## Conclusion


TODO
- Bruno's second wave of feedbacks
    1. Structure 
    2. Change the architecture of the RNN to beat the baseline
        - `recurrent_dropout` within the LSTM layer
        - `dropout` within the LSTM layer
        - regularizing the Dense Layer

- Write the take-aways

## Acknowledgments

* This challenge was truly inspired by the `Time Series Forecasting` tutorial from `Google>Tensorflow>Keras`
* The technical functions were inspired by Bruno's boilerplate about Time Series